In [99]:
# Import required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [100]:
# Get data from wikipedia page, process the data to extract table
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html')
# print(soup.prettify())
postal_codes_table = soup.find('table',{'class':'wikitable'})
# postal_codes_table

In [101]:
# Extracting postal codes and other data
rows = postal_codes_table.findAll('tr')[1:]
postal_codes = []
boroughs = []
neighborhoods = []

for row in rows:
    postal_codes.append(row.findAll('td')[0].renderContents().strip().decode("utf-8") )
    boroughs.append(row.findAll('td')[1].renderContents().strip().decode("utf-8") )
    neighborhoods.append(row.findAll('td')[2].renderContents().strip().decode("utf-8") )
    
# print('postal_codes list: ' + str(postal_codes))
# print('boroughs list: ' + str(boroughs))
# print('neighborhoods list: ' + str(neighborhoods))

In [103]:
# Create a pandas dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = columns) 
df['PostalCode'] = postal_codes
df['Borough'] = boroughs
df['Neighborhood'] = neighborhoods
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [104]:
# Remove all rows with Borough = 'Not assigned'
df.drop(df[df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)

# 
for k in range(1,df.shape[0]):
    neigh = df['Neighborhood'].iloc[k] 
    if neigh.find('/') > 0:
        df['Neighborhood'].iloc[k] = ','.join(df['Neighborhood'].iloc[k].split('/'))
    
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighborhood'] == 'Not assigned']['Neighborhood'] = df.loc[df['Neighborhood'] == 'Not assigned']['Borough']

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [105]:
df.shape

(103, 3)